In [1]:
from nltk.probability import FreqDist
import csv
import pandas as pd
from nltk.metrics import agreement

# Confusion Matrix

In [ ]:
class ConfusionMatrix(object):


    def __init__(self, reference, test, sort_by_count=False):
        """
        Construct a new confusion matrix from a list of reference
        values and a corresponding list of test values.

        :type reference: list
        :param reference: An ordered list of reference values.
        :type test: list
        :param test: A list of values to compare against the
            corresponding reference values.
        :raise ValueError: If ``reference`` and ``length`` do not have
            the same length.
        """
        if len(reference) != len(test):
            raise ValueError("Lists must have the same length.")

        # Get a list of all values.
        if sort_by_count:
            ref_fdist = FreqDist(reference)
            test_fdist = FreqDist(test)

            def key(v):
                return -(ref_fdist[v] + test_fdist[v])

            values = sorted(set(reference + test), key=key)
        else:
            values = sorted(set(reference + test))

        # Construct a value->index dictionary
        indices = dict((val, i) for (i, val) in enumerate(values))

        # Make a confusion matrix table.
        confusion = [[0 for val in values] for val in values]
        max_conf = 0  # Maximum confusion
        for w, g in zip(reference, test):
            confusion[indices[w]][indices[g]] += 1
            max_conf = max(max_conf, confusion[indices[w]][indices[g]])

        #: A list of all values in ``reference`` or ``test``.
        self._values = values
        #: A dictionary mapping values in ``self._values`` to their indices.
        self._indices = indices
        #: The confusion matrix itself (as a list of lists of counts).
        self._confusion = confusion
        #: The greatest count in ``self._confusion`` (used for printing).
        self._max_conf = max_conf
        #: The total number of values in the confusion matrix.
        self._total = len(reference)
        #: The number of correct (on-diagonal) values in the matrix.
        self._correct = sum(confusion[i][i] for i in range(len(values)))

    def __getitem__(self, li_lj_tuple):
        """
        :return: The number of times that value ``li`` was expected and
        value ``lj`` was given.
        :rtype: int
        """
        (li, lj) = li_lj_tuple
        i = self._indices[li]
        j = self._indices[lj]
        return self._confusion[i][j]

    def __repr__(self):
        return "<ConfusionMatrix: %s/%s correct>" % (self._correct, self._total)

    def __str__(self):
        return self.pretty_format()

    def pretty_format(
        self,
        show_percents=False,
        values_in_chart=True,
        truncate=None,
        sort_by_count=False,
    ):
        """
        :return: A multi-line string representation of this confusion matrix.
        :type truncate: int
        :param truncate: If specified, then only show the specified
            number of values.  Any sorting (e.g., sort_by_count)
            will be performed before truncation.
        :param sort_by_count: If true, then sort by the count of each
            label in the reference data.  I.e., labels that occur more
            frequently in the reference label will be towards the left
            edge of the matrix, and labels that occur less frequently
            will be towards the right edge.

        @todo: add marginals?
        """
        confusion = self._confusion

        values = self._values
        if sort_by_count:
            values = sorted(
                values, key=lambda v: -sum(self._confusion[self._indices[v]])
            )

        if truncate:
            values = values[:truncate]

        if values_in_chart:
            value_strings = ["%s" % val for val in values]
        else:
            value_strings = [str(n + 1) for n in range(len(values))]

        # Construct a format string for row values
        valuelen = max(len(val) for val in value_strings)
        value_format = "%" + repr(valuelen) + "s | "
        # Construct a format string for matrix entries
        if show_percents:
            entrylen = 6
            entry_format = "%5.1f%%"
            zerostr = "     ."
        else:
            entrylen = len(repr(self._max_conf))
            entry_format = "%" + repr(entrylen) + "d"
            zerostr = " " * (entrylen - 1) + "."

        # Write the column values.
        s = ""
        for i in range(valuelen):
            s += (" " * valuelen) + " |"
            for val in value_strings:
                if i >= valuelen - len(val):
                    s += val[i - valuelen + len(val)].rjust(entrylen + 1)
                else:
                    s += " " * (entrylen + 1)
            s += " |\n"

        # Write a dividing line
        s += "%s-+-%s+\n" % ("-" * valuelen, "-" * ((entrylen + 1) * len(values)))

        # Write the entries.
        for val, li in zip(value_strings, values):
            i = self._indices[li]
            s += value_format % val
            for lj in values:
                j = self._indices[lj]
                if confusion[i][j] == 0:
                    s += zerostr
                elif show_percents:
                    s += entry_format % (100.0 * confusion[i][j] / self._total)
                else:
                    s += entry_format % confusion[i][j]
                if i == j:
                    prevspace = s.rfind(" ")
                    s = s[:prevspace] + "<" + s[prevspace + 1 :] + ">"
                else:
                    s += " "
            s += "|\n"

        # Write a dividing line
        s += "%s-+-%s+\n" % ("-" * valuelen, "-" * ((entrylen + 1) * len(values)))

        # Write a key
        s += "(row = " + name_1 + "; col = " + name_2 + ")" + "\n"
        if not values_in_chart:
            s += "Value key:\n"
            for i, value in enumerate(values):
                s += "%6d: %s\n" % (i + 1, value)

        return s


    def key(self):
        values = self._values
        str = "Value key:\n"
        indexlen = len(repr(len(values) - 1))
        key_format = "  %" + repr(indexlen) + "d: %s\n"
        for i in range(len(values)):
            str += key_format % (i, values[i])

        return str

## Confusion Matrix

In [2]:
df = pd.read_csv('../data/inter-annotator/annotator_agreement_all_final.csv', sep = ';')

In [3]:
df.head(5)

,Headlines,vlad_entity,vlad,shanet_entity,shanet,dhruv_entity,dhruv,spacy_entity,spacy
0,This Land Was Made For You And Me' — Or Was It?,land,LOC,Land,LOC,Land,LOC,This Land Was Made,WORK_OF_ART
1,A Perspective On George Zimmerman That Every P...,George Zimmerman,PERSON,George Zimmerman,PERSON,George Zimmerman,PERSON,George Zimmerman,PERSON
2,A Strategy To Combat Crime That McGruff Wished...,McGruff,PERSON,McGruff,PERSON,McGruff,PERSON,McGruff,PERSON
3,Could You Go 100 Days Without Food? These Men ...,100 Days,DATE,Men,PERSON,100 Days,DATE,100 Days,DATE
4,Here’s A Map of American Exceptionalism That’s...,American Exceptionalism,NORP,American,NORP,American,NORP,American,NORP


In [ ]:
# name_1 = "Vlad"
name_1 = "Shanet"
# name_1 = "Dhruv"
# name_2 = "Shanet"
# name_2 = "Dhruv"
name_2 = "Spacy"

def demo():
    list1=[]
    list2=[]

    with open('../data/inter-annotator/annotator_agreement_all_final.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=';')
        line_count = 0

        for row in csv_reader:
            # VLAD
            #list1.append(row[2])
            # SHANET
            #list1.append(row[4])
            # DHRUV
            list1.append(row[6])
            # SPACY
            list2.append(row[8])
            line_count += 1
        print(f'Processed {line_count-1} lines.')
    a1=list1[1:]
    a2=list2[1:]

    print(name_1, len(list1[1:]))
    print(name_2, len(list2[1:]))
    print("Confusion matrix:")

    print(ConfusionMatrix(a1, a2).pretty_format(sort_by_count=True))

if __name__ == "__main__":
    demo()

## Kappa score for annotators

In [ ]:
df.head()

In [ ]:
inputfile='../data/inter-annotator/annotator_agreement_all_final.csv'
print(inputfile)
merged_df=pd.read_csv(inputfile, delimiter=';', header = 0, index_col=0)
merged_df.columns=["vlad_entity", "vlad", "shanet_entity", "shanet", "dhruv_entity", "dhruv", "spacy_entity", "spacy"]

print(merged_df.shape)

labels_matched_df = merged_df.dropna()

# Reformat the data into the form AnnotationTask
#  expects.
data = []
for idx, row in labels_matched_df.iterrows():
    data.append(("a1", idx, row["shanet"]))
    data.append(("a2", idx, row["vlad"]))

atask = agreement.AnnotationTask(data=data)

print("Percentage agreement:", atask.avg_Ao())
print("Cohen's Kappa:", atask.kappa())
#print("Fleiss's Kappa:", atask.multi_kappa())
#print("Krippendorf's alpha:", atask.alpha())

# This function maps labels into a numeric space,
# . so we can rely on the ordering of labels.
def priority_distance(left_label, right_label):
    mapped_labels = {
        "Critical": 4,
        "High": 3,
        "Medium": 2,
        "Low": 1,
    }
    left_i = mapped_labels[left_label]
    right_i = mapped_labels[right_label]

    return abs(left_i - right_i)

# Extracting disagreements
## ANNOTATORS - SPACY

In [4]:
df.head()

,Headlines,vlad_entity,vlad,shanet_entity,shanet,dhruv_entity,dhruv,spacy_entity,spacy
0,This Land Was Made For You And Me' — Or Was It?,land,LOC,Land,LOC,Land,LOC,This Land Was Made,WORK_OF_ART
1,A Perspective On George Zimmerman That Every P...,George Zimmerman,PERSON,George Zimmerman,PERSON,George Zimmerman,PERSON,George Zimmerman,PERSON
2,A Strategy To Combat Crime That McGruff Wished...,McGruff,PERSON,McGruff,PERSON,McGruff,PERSON,McGruff,PERSON
3,Could You Go 100 Days Without Food? These Men ...,100 Days,DATE,Men,PERSON,100 Days,DATE,100 Days,DATE
4,Here’s A Map of American Exceptionalism That’s...,American Exceptionalism,NORP,American,NORP,American,NORP,American,NORP


## 1 Shanet - spaCy

In [5]:
result = df[(df['shanet'] == 'PRODUCT') & (df['spacy'] == 'WORK_OF_ART')]

In [6]:
result[['Headlines', 'shanet', 'shanet_entity', 'spacy', 'spacy_entity']]

,Headlines,shanet,shanet_entity,spacy,spacy_entity
21,"""Damn It."" -The Present. ""Sorry About That!"" -...",PRODUCT,Present,WORK_OF_ART,Sorry About That
23,"""Hum De Dum Dum, Singing In The Shower, Hum De...",PRODUCT,The Shower,WORK_OF_ART,"Hum De Dum Dum, Singing In The Shower, Hum De ..."
32,"""Seeing This Picture Reminds Me Of Everything ...",PRODUCT,Picture,WORK_OF_ART,Seeing This Picture Reminds
39,"""Unhand Him You Foul Beast!"" Is What Looking A...",PRODUCT,Cartoon,WORK_OF_ART,Unhand Him


In [7]:
result2 = df[(df['shanet'] == 'LOC') & (df['spacy'] == 'WORK_OF_ART')]
result2[['Headlines', 'shanet', 'shanet_entity', 'spacy', 'spacy_entity']]

,Headlines,shanet,shanet_entity,spacy,spacy_entity
0,This Land Was Made For You And Me' — Or Was It?,LOC,Land,WORK_OF_ART,This Land Was Made
25,"""I'm Just A Bill"" Parody A.K.A. Schoolhouse Co...",LOC,Schoolhouse,WORK_OF_ART,I'm Just A Bill


## 2 Dhruv - spaCY

In [8]:
result_dhruv = df[(df['dhruv'] == 'DATE') & (df['spacy'] == 'WORK_OF_ART')]
result_dhruv[['Headlines', 'dhruv', 'dhruv_entity', 'spacy', 'spacy_entity']]

,Headlines,dhruv,dhruv_entity,spacy,spacy_entity
21,"""Damn It."" -The Present. ""Sorry About That!"" -...",DATE,The Past,WORK_OF_ART,Sorry About That
40,"""Women Just Haven't Done That Much"" Could Be M...",DATE,2013,WORK_OF_ART,Women Just Haven't Done That Much


In [9]:
value = result_dhruv.Headlines
value.iloc[0]

'"Damn It." -The Present. "Sorry About That!" -The Past.'

In [10]:
value.iloc[1]

'"Women Just Haven\'t Done That Much" Could Be My Least Favorite Media Moment Of 2013. What\'s Yours?'

In [11]:
result_dhruv2 = df[(df['dhruv'] == 'NORP') & (df['spacy'] == 'ORG')]
result_dhruv2[['Headlines', 'dhruv', 'dhruv_entity', 'spacy', 'spacy_entity']]

,Headlines,dhruv,dhruv_entity,spacy,spacy_entity
29,"""Prohibition Worked For Alcohol, It Can Work F...",NORP,Congress,ORG,Congress
30,"""STRAIGHT TO THE SOURCE: If You Have Questions...",NORP,Islam,ORG,Islam


In [12]:
value = result_dhruv2.Headlines
value.iloc[0]

'"Prohibition Worked For Alcohol, It Can Work For Drugs Too!" Congress'

In [13]:
value.iloc[1]

'"STRAIGHT TO THE SOURCE: If You Have Questions About Islam, Maybe Ask A Muslim?'

## 3 Vlad - spaCY

In [14]:
result_vlad = df[(df['vlad'] == 'PERSON') & (df['spacy'] == "WORK_OF_ART")]
result_vlad[['Headlines', 'vlad', 'vlad_entity', 'spacy', 'spacy_entity']]

,Headlines,vlad,vlad_entity,spacy,spacy_entity
38,"""Uncle Johnny Stews Good Time Syria Jamboree"" ...",PERSON,Uncle Johnny,WORK_OF_ART,Uncle Johnny Stews Good Time Syria Jamboree
44,"""You throw like a girl!"" MythBusters puts the ...",PERSON,Mythin Busters,WORK_OF_ART,MythBusters
46,#Brozac: The Solution For When The Media Stops...,PERSON,Brozac,WORK_OF_ART,Brozac


In [15]:
value = result_vlad.Headlines
value.iloc[0]

'"Uncle Johnny Stews Good Time Syria Jamboree" Plays The Right Tune About Our Involvement'

In [16]:
value.iloc[1]

'"You throw like a girl!" MythBusters puts the classic insult to the test.'

In [17]:
value.iloc[2]

'#Brozac: The Solution For When The Media Stops Catering To You. The Struggle Is Real.'

In [18]:
result_vlad2 = df[(df['vlad'] == 'LOC') & (df['spacy'] == "WORK_OF_ART")]
result_vlad2[['Headlines', 'vlad', 'vlad_entity', 'spacy', 'spacy_entity']]

,Headlines,vlad,vlad_entity,spacy,spacy_entity
0,This Land Was Made For You And Me' — Or Was It?,LOC,land,WORK_OF_ART,This Land Was Made
23,"""Hum De Dum Dum, Singing In The Shower, Hum De...",LOC,the shower,WORK_OF_ART,"Hum De Dum Dum, Singing In The Shower, Hum De ..."


In [19]:
value = result_vlad2.Headlines
value.iloc[0]

"This Land Was Made For You And Me' — Or Was It?"

In [20]:
value.iloc[1]

'"Hum De Dum Dum, Singing In The Shower, Hum De Dum Du...oh, Oh NO! NOOO!... Oh, How Considerate!"'

## ANNOTATOR - ANNOTATOR

## Shanet - Vlad

In [21]:
shanet_vlad = df[(df['shanet'] == 'ORG') & (df['vlad'] == "PERSON")]
shanet_vlad2 = df[(df['shanet'] == 'PERSON') & (df['vlad'] == "WORK_OF_ART")]
shanet_vlad[['Headlines', 'shanet', 'shanet_entity', 'vlad', 'vlad_entity']]

,Headlines,shanet,shanet_entity,vlad,vlad_entity
44,"""You throw like a girl!"" MythBusters puts the ...",ORG,MythBustera,PERSON,Mythin Busters
46,#Brozac: The Solution For When The Media Stops...,ORG,Brozac,PERSON,Brozac


In [22]:
shanet_vlad2[['Headlines', 'shanet', 'shanet_entity', 'vlad', 'vlad_entity']]

,Headlines,shanet,shanet_entity,vlad,vlad_entity
40,"""Women Just Haven't Done That Much"" Could Be M...",PERSON,Women,WORK_OF_ART,Woman just haven't done much


In [23]:
for index, row in shanet_vlad.iterrows():
    print('1 - ', row['Headlines'])
    
value = shanet_vlad2.Headlines
print('2 - ', value.iloc[0])

1 -  "You throw like a girl!" MythBusters puts the classic insult to the test.
1 -  #Brozac: The Solution For When The Media Stops Catering To You. The Struggle Is Real.
2 -  "Women Just Haven't Done That Much" Could Be My Least Favorite Media Moment Of 2013. What's Yours?


## Vlad - Dhruv

In [24]:
vlad_dhruv = df[(df['vlad'] == 'PERSON') & (df['dhruv'] == "NORP")]
vlad_dhruv2 = df[(df['vlad'] == 'PERSON') & (df['dhruv'] == "ORG")]
vlad_dhruv[['Headlines', 'vlad', 'vlad_entity', 'dhruv', 'dhruv_entity']]

,Headlines,vlad,vlad_entity,dhruv,dhruv_entity
16,"""A major American theater has cast a black wom...",PERSON,Hamlet,NORP,American
30,"""STRAIGHT TO THE SOURCE: If You Have Questions...",PERSON,Muslim,NORP,Islam


In [26]:
for index, row in vlad_dhruv2.iterrows():
    print('1 - ', row['Headlines'])

1 -  The President Of Brazil Is Going The George Bush Route For This Year's World Cup.
1 -  #Brozac: The Solution For When The Media Stops Catering To You. The Struggle Is Real.


In [27]:
vlad_dhruv2[['Headlines', 'vlad', 'vlad_entity', 'dhruv', 'dhruv_entity']]

,Headlines,vlad,vlad_entity,dhruv,dhruv_entity
8,The President Of Brazil Is Going The George Bu...,PERSON,the president of Brazil,ORG,President
46,#Brozac: The Solution For When The Media Stops...,PERSON,Brozac,ORG,The Media


## Dhruv - Shanet

In [28]:
dhruv_shanet = df[(df['dhruv'] == 'WORK_OF_ART') & (df['shanet'] == "ORG")]
dhruv_shanet2 = df[(df['dhruv'] == 'WORK_OF_ART') & (df['shanet'] == "PRODUCT")]
dhruv_shanet[['Headlines', 'dhruv', 'dhruv_entity', 'shanet', 'shanet_entity']]

,Headlines,dhruv,dhruv_entity,shanet,shanet_entity
26,"""I've Got Nothing To Hide"" Is The Wrong Reacti...",WORK_OF_ART,I've Got Nothing To Hide,ORG,NSA
44,"""You throw like a girl!"" MythBusters puts the ...",WORK_OF_ART,MythBusters,ORG,MythBustera


In [32]:
for index, row in dhruv_shanet.iterrows():
    print('1 - ', row['Headlines'])

1 -  "I've Got Nothing To Hide" Is The Wrong Reaction To Have To The NSA Scandal. Here's Why.
1 -  "You throw like a girl!" MythBusters puts the classic insult to the test.


In [29]:
dhruv_shanet2[['Headlines', 'dhruv', 'dhruv_entity', 'shanet', 'shanet_entity']]

,Headlines,dhruv,dhruv_entity,shanet,shanet_entity
23,"""Hum De Dum Dum, Singing In The Shower, Hum De...",WORK_OF_ART,Singing In The Shower,PRODUCT,The Shower
32,"""Seeing This Picture Reminds Me Of Everything ...",WORK_OF_ART,This Picture,PRODUCT,Picture


In [31]:
for index, row in dhruv_shanet2.iterrows():
    print('1 - ', row['Headlines'])

1 -  "Hum De Dum Dum, Singing In The Shower, Hum De Dum Du...oh, Oh NO! NOOO!... Oh, How Considerate!"
1 -  "Seeing This Picture Reminds Me Of Everything I Went Through"
